# Proxy Pattern

- __Type:__ Structural
- __Popularity: ★★★★☆__
- __Complexity: ★★★☆☆__

## Intent

The Proxy pattern provides a surrogate or placeholder for another object to control access to it. It allows you to:

- Control access to the original object
- Add functionality when accessing the object
- Delay the creation and initialization of expensive objects until needed
- Implement access control for the original object's clients

## Problem

Sometimes direct access to an object isn't possible or desirable for various reasons:

- The object might be expensive to create or use (memory, computation, etc.)
- The object might require access control (authorization, authentication)
- The object might be remote, and network operations need to be managed
- You might need additional behavior before/after accessing the real object

Without the Proxy pattern, clients would need to manage these concerns themselves, leading to code duplication and tight coupling between clients and the service objects.

## Solution

The Proxy pattern suggests creating a new proxy class with the same interface as the original service object. The proxy maintains a reference to the service object and controls access to it by:

1. Creating the service object only when it's really needed (lazy initialization)
2. Checking access permissions before forwarding requests
3. Logging requests or performing other tasks before/after forwarding
4. Managing remote service interaction details (network communication)

The proxy acts as a substitute for the real service object, allowing it to perform additional operations before or after passing the request to the original object.

## Diagram

```mermaid
classDiagram
    class Subject {
        <<interface>>
        +request()
    }
    class RealSubject {
        +request()
    }
    class Proxy {
        -realSubject: RealSubject
        +request()
    }
    
    Subject <|.. RealSubject
    Subject <|.. Proxy
    Proxy o-- RealSubject
```

- **Subject**: The interface that both the RealSubject and Proxy implement
- **RealSubject**: The actual object that does the real work
- **Proxy**: Holds a reference to the RealSubject and controls access to it

## Example 1: Basic Proxy Pattern

In [1]:
from abc import ABC, abstractmethod


class Subject(ABC):
    """Interface implemented by both RealSubject and Proxy"""

    @abstractmethod
    def request(self) -> None:
        pass


class RealSubject(Subject):
    """The real object that the proxy represents"""

    def request(self) -> None:
        print("RealSubject: Handling request.")


class Proxy(Subject):
    """The Proxy maintains a reference to the RealSubject and controls access to it"""

    def __init__(self, real_subject: RealSubject) -> None:
        self._real_subject = real_subject

    def check_access(self) -> bool:
        print("Proxy: Checking access prior to firing a real request.")
        return True

    def log_access(self) -> None:
        print("Proxy: Logging the request.")

    def request(self) -> None:
        """The proxy adds functionality before and after delegating to the real subject"""
        if self.check_access():
            self._real_subject.request()
            self.log_access()

### Usage

In [2]:
def client_code(subject: Subject) -> None:
    """Client code that works with all objects (both the real and proxy ones)"""
    subject.request()


if __name__ == "__main__":
    print("Client: Executing the client code with a real subject:")
    real_subject = RealSubject()
    client_code(real_subject)

    print("\nClient: Executing the same client code with a proxy:")
    proxy = Proxy(real_subject)
    client_code(proxy)

Client: Executing the client code with a real subject:
RealSubject: Handling request.

Client: Executing the same client code with a proxy:
Proxy: Checking access prior to firing a real request.
RealSubject: Handling request.
Proxy: Logging the request.


## Example 2: Protection (Access Control) Proxy

In [3]:
class SensitiveInfo:
    """Contains sensitive information that needs protection"""

    def __init__(self):
        self.users = {"admin": "admin_password", "user": "user_password"}

    def read(self):
        print("Reading sensitive information: ", self.users)

    def add(self, user, password):
        self.users[user] = password
        print(f"Added user: {user}")


class SensitiveInfoProxy:
    """Acts as a protection proxy for SensitiveInfo"""

    def __init__(self, username, password):
        self.sensitive_info = SensitiveInfo()
        self.username = username
        self.password = password

    def read(self):
        if self._authenticate():
            if self._authorize_read():
                self.sensitive_info.read()
            else:
                print("Authorization failed: Cannot read sensitive info")
        else:
            print("Authentication failed")

    def add(self, user, password):
        if self._authenticate():
            if self._authorize_write():
                self.sensitive_info.add(user, password)
            else:
                print("Authorization failed: Cannot modify sensitive info")
        else:
            print("Authentication failed")

    def _authenticate(self):
        stored_password = self.sensitive_info.users.get(self.username)
        return stored_password == self.password

    def _authorize_read(self):
        # All authenticated users can read
        return True

    def _authorize_write(self):
        # Only admin can modify
        return self.username == "admin"

### Usage

In [4]:
if __name__ == "__main__":
    # Admin can both read and write
    print("Admin operations:")
    admin_proxy = SensitiveInfoProxy("admin", "admin_password")
    admin_proxy.read()
    admin_proxy.add("new_user", "new_password")

    print("\nRegular user operations:")
    # Regular user can read but not write
    user_proxy = SensitiveInfoProxy("user", "user_password")
    user_proxy.read()
    user_proxy.add("another_user", "password123")  # Will fail

    print("\nInvalid credentials:")
    # Invalid credentials
    invalid_proxy = SensitiveInfoProxy("user", "wrong_password")
    invalid_proxy.read()  # Will fail

Admin operations:
Reading sensitive information:  {'admin': 'admin_password', 'user': 'user_password'}
Added user: new_user

Regular user operations:
Reading sensitive information:  {'admin': 'admin_password', 'user': 'user_password'}
Authorization failed: Cannot modify sensitive info

Invalid credentials:
Authentication failed


## Example 3: Virtual Proxy (Lazy Loading)

In [5]:
class ExpensiveObject:
    """Simulates an expensive object to create"""

    def __init__(self):
        self._heavy_data = None
        self._initialize()

    def _initialize(self):
        import time

        print("Creating expensive object...")
        time.sleep(2)  # Simulate heavy initialization
        self._heavy_data = [i for i in range(10000)]
        print("Expensive object created")

    def process(self):
        print(f"Processing data with {len(self._heavy_data)} items...")


class VirtualProxy:
    """Proxy that delays creation of the expensive object until needed"""

    def __init__(self):
        self._real_object = None

    def process(self):
        # Create the object only when needed
        if self._real_object is None:
            self._real_object = ExpensiveObject()
        self._real_object.process()

### Usage

In [6]:
if __name__ == "__main__":
    print("Creating proxy object (no expensive object created yet)...")
    proxy = VirtualProxy()
    print("Proxy created, but expensive object not created yet.")

    print("\nCalling process for the first time:")
    proxy.process()  # This will create the expensive object

    print("\nCalling process again (expensive object already created):")
    proxy.process()  # This will reuse the existing expensive object

Creating proxy object (no expensive object created yet)...
Proxy created, but expensive object not created yet.

Calling process for the first time:
Creating expensive object...
Expensive object created
Processing data with 10000 items...

Calling process again (expensive object already created):
Processing data with 10000 items...


## Real-world analogies

1. **Credit Card**:
   A credit card is a proxy for a bank account. It provides the same interface (payment) but adds functionality like transaction limits, fraud detection, and delayed actual payment.

2. **Administrative Assistant**:
   An assistant controls access to a busy executive, screening calls, arranging meetings, and filtering information. The assistant acts as a proxy to the executive, handling routine matters directly and forwarding only important matters.

3. **Web Cache**:
   A web cache stores copies of resources (like images, pages) from the internet, serving as a proxy between users and the remote web servers. It improves performance by delivering content locally instead of fetching it repeatedly.

## When to use

- **Lazy initialization** (Virtual Proxy): When the real object is expensive to create and might not be used
- **Access control** (Protection Proxy): When you need to control who and when can access the real object
- **Logging/Auditing** (Logging Proxy): When you need to keep a record of requests to the real object
- **Caching** (Caching Proxy): When the results of expensive operations need to be reused
- **Remote resource access** (Remote Proxy): When the real object is located on a remote server
- **Smart references**: When you need to add additional actions when an object is accessed or destroyed

## Python-specific implementation notes

- In Python, the `__getattr__` and `__setattr__` magic methods can be used to create dynamic proxies
- Python's dynamic nature means you can often create proxies without explicitly implementing all interface methods
- The standard library's `functools.lru_cache` decorator acts as a caching proxy for function calls
- Python's property decorators (`@property`) can serve as a simplified form of proxy pattern
- The `multiprocessing.Manager` objects in Python act as proxies for sharing data between processes
- Libraries like `SQLAlchemy` use proxies to manage database connections and transactions

## Related patterns

- **Adapter Pattern**: While Proxy provides the same interface, Adapter provides a different interface to the wrapped object
- **Decorator Pattern**: Adds responsibilities to objects without subclassing, focuses on adding behavior rather than controlling access
- **Facade Pattern**: Provides a simplified interface to a complex subsystem rather than controlling access to an object
- **Singleton Pattern**: Often used with Proxy to ensure only one instance of the real subject exists
- **Factory Method**: Can be used to create proxy objects without exposing the real object's instantiation logic